# Chapitre 4.3 - Les fichiers CSV, JSON et les requêtes

---

## 1. La structure du web : comment communique-t-on avec un serveur ?

![Anatomie d'une communication HTTP](images/http.request.scheme.png)

Lors d'une communication HTTP avec un serveur, la communication est divisible en deux : l'envoi de la requête et la réponse du serveur. Ces deux éléments de la communication répondent à un ensemble de standards très stricts permettant le fonctionnement du web tel que nous le connaissons.

### 1.A. Anatomie d'une requête :

![Anatomie d'une requête HTTP](images/http.request.request.png)

La requête, c'est-à-dire l'information envoyée au serveur, est composée à minima de trois types d'informations :

- l'URL
- la méthode
- les headers

#### URL

Voir le [schéma](images/url.png) ([Source](https://cascadingmedia.com/assets/images/insites/2015/02/url-anatomy/url-anatomy-55598c24.png) 
L'URL est une information que l'on connait tous. C'est l'adresse dont on requiert le contenu. Typiquement, l'adresse est divisible en plusieurs parties. Celle qui peut être importante et qui changera surement suivant les utilisateurs est la partie *query* qui permet d'apporter des informations supplémentaires.

Par exemple, dans http://cts.dh.uni-leipzig.de/api/cts?request=GetCapabilities&urn=urn:cts:latinLit:phi1294 , on a deux paramètres fournis :

| Nom | Valeur |
| --- | ------ |
| urn | urn:cts:latinLit:phi1294 |
| request | GetCapabilities |

#### Méthode

La méthode informe le serveur de ce que vous allez vouloir faire. 90% des requêtes que vous faites en naviguant sur le web sont en GET : vous récupérez de l'information. Vous utilisez sur les 9.9% restant la requête POST, notamment quand vous vous connectez sur vos comptes sur les divers sites que vous utilisez.

#### Les Headers

Le Header comporte des informations sur vos attentes et votre contexte de requêtage. Par exemple, on peut demander via les Headers un format de réponse particulier (d'après son [mimetype](https://fr.wikipedia.org/wiki/Type_MIME) : html, xml ou json par exemple : 

| Headers Clé | Headers Valeur   |
| ----------- | ---------------- |
| Accept      | application/json |

#### (Optionnel) Le Corps (Body, data, etc.)

Dans le cadre de l'envoi d'un formulaire ou d'un fichier, on a un corps dans la requête. Beaucoup de formats différents sont possibles dans ce cadre. De nombreuses API acceptent par exemple l'encodage en JSON de vos informations.

### 1.B. Anatomie d'une réponse

![Anatomie d'une réponse HTTP](images/http.request.response.png)

La réponse est composée de trois éléments aussi :

#### Les Headers 

Tout comme la requête, les Headers nous renvoient l'information sur la réponse. Voici quelques headers utiles.


| Headers Clé | Headers Valeur   | Note |
| ----------- | ---------------- | ---- |
| Encoding      | application/json | Type Mime de la réponse |

#### Le code HTTP

Le code HTTP nous informe sur le statut de la réponse. Vous connaissez *à minima* le code 404, qui indique que la ressource demandée n'est pas disponible. Il existe bien d'autres codes (*cf.* [Wikipedia](https://fr.wikipedia.org/wiki/Liste_des_codes_HTTP) :
- 200 : succès de la requête ;
- 301 et 302 : redirection, respectivement permanente et temporaire ;
- 401 : utilisateur non authentifié ;
- 403 : accès refusé ;
- 404 : page non trouvée ;
- 500 et 503 : erreur serveur.
- 418 : "I’m a teapot" (Blague du 1er avril 1998 restée dans le standard)



#### Le Corps

Le corps de la réponse contient bien évidemment ce que vous voyez lorsque vous faites une requête : le contenu html, le contenu en plein texte, le contenu json, etc.

## 2. Faire des requêtes http en python : le module request


### 2.A Le module `requests`


Le module `requests` possède sa documentation sur son [propre site](http://docs.python-requests.org/en/master/). Au moment de l'écriture, le module est dans sa version 2.18.4.

### 2.B Faire une requête GET:

Le module est très simple : il propose une fonction `get` qui prend une URL !

In [1]:
import requests

# Pour l'exemple nous utilisons l'API de Chronicling America, un projet de numérisation de journaux
# Américains
req = requests.get("https://chroniclingamerica.loc.gov/search/pages/results/?format=json&proxtext=ecole+nationale")
print(req)

<Response [200]>


Les objets `Response` ont plusieurs propriétés intéressantes : 

- `.status_code` sous la forme d'un entier qui informe du succès de la requête
- `.headers` sous la forme d'un dictionnaire qui comporte l'ensemble des headers
- `.encoding` qui comprend la méthode d'encodage
- `.text` qui contient le contenu de la réponse
- `.json()` qui, si `.headers['content-type']` est `application/json`, parse lui-même le json de la réponse. 

Voyons un peu leur contenu :

In [4]:
print(req.status_code)
print(req.headers)
print(req.encoding)

{'Date': 'Fri, 17 Dec 2021 13:11:56 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Last-Modified': 'Fri, 17 Dec 2021 13:11:56 GMT', 'Expires': 'Sat, 18 Dec 2021 13:11:56 GMT', 'Cache-Control': 's-maxage=86400, public, max-age=86400', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'X-requested-with', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Varnish': '31263324', 'Via': '1.1 varnish (Varnish/5.2)', 'CF-Cache-Status': 'MISS', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '6bf06641cc36021d-ZRH'}


In [5]:
# Puisque l'on a du json, on peut le traiter comme un dictionnaire ou une liste
# Une rapide ouverture de la page m'informe que le nom du journal est disponible à la clé ""

for resultat in req.json()["items"]:
    print("\"{titre}\" a publié un article comprenant 'école nationale' le {jour}/{mois}/{annee}".format(
        titre=resultat["title_normal"], 
        annee=resultat["date"][:4], 
        mois=resultat["date"][4:6],
        jour=resultat["date"][6:]
    ))

"arizona post." a publié un article comprenant 'école nationale' le 11/01/1963
"mississippi enterprise." a publié un article comprenant 'école nationale' le 16/10/1954
"evening star." a publié un article comprenant 'école nationale' le 27/04/1930
"washington times." a publié un article comprenant 'école nationale' le 13/11/1921
"evening star." a publié un article comprenant 'école nationale' le 26/06/1932
"evening star." a publié un article comprenant 'école nationale' le 10/04/1932
"evening star." a publié un article comprenant 'école nationale' le 28/06/1931
"jackson advocate." a publié un article comprenant 'école nationale' le 09/10/1954
"auttaja." a publié un article comprenant 'école nationale' le 22/11/1951
"sistersville daily oil review." a publié un article comprenant 'école nationale' le 16/09/1902
"santa fe new mexican." a publié un article comprenant 'école nationale' le 13/01/1910
"wenatchee daily world." a publié un article comprenant 'école nationale' le 10/01/1910
"even

La construction d'URL pouvant poser des problèmes (échappement de caractère par exemple), `requests.get()` accepte un paramètre `params`:

In [ ]:
req = requests.get("http://cts.dh.uni-leipzig.de/api/cts", params={
    "urn": "urn:cts:latinLit:phi1294.phi002.perseus-lat2:1.pr.1",
    "request": "GetPassage"
})
print("URL : " + req.url)
if req.status_code == 200:
    print(req.text)

De la même manière, la méthode `.get()` accepte des headers sous la forme d'un dictionnaire :

In [ ]:
import requests

# L'adresse suivante permet de demander l'analyse morphologique d'un terme :
url = "http://morph.alpheios.net/api/v1/analysis/word?word=lasciva&lang=lat&engine=whitakerLat"

xml = requests.get(url, headers={"Accept": "text/xml"})
print(xml.text)
req_json = requests.get(url, headers={"Accept": "application/json"})
print(req_json.text)

### 2.C Les autres types de requêtes :

Le module possède de la même manière une méthode `requests.post()` qui prendra en plus un paramètre `data` tout comme il possède les méthodes :

- `.update()`
- `.delete()`
- `.put()`
- `.options()`

Toutes ces requêtes prennent les mêmes paramètres que `.get()`

### 2.D Générer une erreur

Imaginons que vous avez un code 404. Vous voulez peut-être éviter de faire tourner un script si cela arrive. L'objet `Response` possède une méthode utile en ce cas : le `.raise_for_status()` :

In [ ]:
import requests 
bad_r = requests.get("http://cts.dh.uni-leipzig.de/collections/urn:cts:froLit")
bad_r.raise_for_status()

----

#### Ce que l'on a appris

Pour finir cette section, voici un récapitulatif des concepts appris. Lisez la liste et posez des questions si certaines choses ne sont pas claires.

- la structure d'une requête http et de sa réponse
- `requests.get`
- `requests.post` et les autres
- `Response.json()`
- `Response.status_code`
- `Response.text`
- `Response.headers`
- `Response.raise_for_status()`